In [1]:
from collections import defaultdict
import json as json
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import numpy as np
import dill

In [ ]:
# This is to save progress file to pkd
#dill.dump(df_2013, open('df_2013.pkd', 'wb'))

In [168]:
dill.dump(df_process, open('df_process.pkd', 'wb'))

In [ ]:
# This is to load pkd file into data frame again
#with open('df_2013_w.pkd', 'rb') as f:
#    df_2013_work = dill.load(f)

In [ ]:
#Test code
#Update "Not Available" string in the data frame into "NaN".
#With "NaN", I can then use scikit learn fill model or 
df['a']=df['a'].apply(lambda x: np.nan if x == 'Not Available' else x)
# to check if there any row is null
df_col.loc[df_col['Electricity Use - Grid Purchase (kBtu)'].isnull()]

#test if still "not available" exists
df_work.loc[df_work['Longitude']=='Not Available']

In [61]:
dill.dump(df_2013_work, open('df_2013_w.pkd', 'wb'))

In [2]:
# This is to load pkd file into data frame again
with open('df_2013_w.pkd', 'rb') as f:
       df_2013_work = dill.load(f)

In [67]:
df_2013 = pd.read_csv("raw data/Energy_and_Water_Data_Disclosure_for_Local_Law_84__Data_for_Calendar_Year_2013_.csv")

C:\Users\rayno\AppData\Local\Temp\ipykernel_6708\410055799.py:1: DtypeWarning: Columns (13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2013 = pd.read_csv("raw data/Energy_and_Water_Data_Disclosure_for_Local_Law_84__Data_for_Calendar_Year_2013_.csv")


In [69]:
# get a list of column names:
cl_name_list = df_2013.columns

Index(['Property Id', 'Property Name', 'Parent Property Id',
       'Parent Property Name', 'Year Ending',
       'NYC Building Identification Number (BIN)',
       'NYC Borough, Block and Lot (BBL)', 'Custom Property ID 1 - Name',
       'Custom Property ID 1 - ID', 'Address 1',
       ...
       'Hotel - Percent That Can Be Cooled', 'Generation Date', 'Release Date',
       'Borough', 'Latitude', 'Longitude', 'Community Board',
       'Council District', 'Census Tract', 'NTA'],
      dtype='object', length=255)

In [73]:
#hand input to pick useful columns

#columns to be used for this capstone project
cl_to_use_list = ['Property Id','City','Primary Property Type - Self Selected','Property Floor Area (Building(s)) (ft²)',
                 'Year Built','ENERGY STAR Score','ENERGY STAR Certification - Eligibility','Latitude','Longitude',
                  'Electricity Use - Grid Purchase (kBtu)',
                  'Office - Computer Density (Number per 1,000 ft²)','Office - Weekly Operating Hours','Office - Worker Density (Number per 1,000 ft²)',
                  'Multifamily Housing - Maximum Number of Floors','Multifamily Housing - Total Number of Residential Living Units','Multifamily Housing - Percent That Can Be Cooled',
                  'Hotel - Room Density (Number per 1,000 ft²)','Hotel - Worker Density (Number per 1,000 ft²)','Hotel - Percent That Can Be Cooled'
                 ]

In [74]:
df_2013_work = df_2013[cl_to_use_list]

In [70]:
df_col = df_2013[cl_to_use_list]

In [82]:
df_2013_work = df_2013_work.rename(columns = {'Property Floor Area (Building(s)) (ft²)':'Total Floor Area -SF',
                                              'Office - Computer Density (Number per 1,000 ft²)':'Office - Computer Density',
                                             'Primary Property Type - Self Selected':'Primary Property Type',
                                             'Hotel - Room Density (Number per 1,000 ft²)':'Hotel Room Density',
                                             'Hotel - Worker Density (Number per 1,000 ft²)':'Hotel Worker Density',
                                             'Hotel - Percent That Can Be Cooled':'Hotel Perecent Area Cooled',
                                             'Office - Worker Density (Number per 1,000 ft²)':'Office - Worker Density',
                                              'Multifamily Housing - Maximum Number of Floors':'Multi Fami Max Floors',
                                              'Multifamily Housing - Total Number of Residential Living Units':'Multi Fami Resid Units',
                                              'Multifamily Housing - Percent That Can Be Cooled':'Multi Fami Area Cooled'
                                             })

In [76]:
# Drop row that doesn't have electricity use (can't be use as Y)
# convert KBTU to KWH
df_2013_work = df_2013_work[df_2013_work['Electricity Use - Grid Purchase (kBtu)'] != 'Not Available']


In [77]:
df_2013_work['Electricity Use - Grid Purchase (kBtu)'] = pd.to_numeric(df_2013_work['Electricity Use - Grid Purchase (kBtu)'], errors='coerce')

In [78]:
df_2013_work['Electricity Use - Grid Purchase (kwh)'] = df_2013_work['Electricity Use - Grid Purchase (kBtu)']*0.000293

In [79]:
df_2013_work['Electricity Use - Grid Purchase (kwh)']

0        12752.849348
1         4835.318378
2         7196.573764
3         5532.209004
4        30763.814288
             ...     
13127      180.366434
13128      326.263499
13130     1895.687996
13131      675.632070
13132     1027.508076
Name: Electricity Use - Grid Purchase (kwh), Length: 12014, dtype: float64

In [84]:
df_work = df_2013_work

In [87]:
# to change all columns with "not available" into 'nan'
cl_not_available = ['ENERGY STAR Score', 'Electricity Use - Grid Purchase (kwh)',
                    'Total Floor Area -SF','Office - Computer Density','Office - Weekly Operating Hours','Office - Worker Density',
                   'Multi Fami Max Floors','Multi Fami Resid Units','Multi Fami Area Cooled',
                   'Hotel Room Density','Hotel Worker Density','Hotel Perecent Area Cooled']


def turn_notavailable(x):

    if x == 'Not Available':
        x=np.nan
    return x

for col in cl_not_available:
    df_work[col] = df_work[col].apply(turn_notavailable)

In [88]:
# to change all columns with dtype 'object' into 'int' or 'float'
cl_to_numeric = ['Total Floor Area -SF','ENERGY STAR Score',
                    'Office - Computer Density','Office - Weekly Operating Hours','Office - Worker Density',
                   'Multi Fami Max Floors','Multi Fami Resid Units','Multi Fami Area Cooled',
                   'Hotel Room Density','Hotel Worker Density','Hotel Perecent Area Cooled']

for col in cl_to_numeric:
    df_work[col] = pd.to_numeric(df_work[col], errors='coerce')

In [94]:
df_work.head(5)

,Property Id,City,Primary Property Type,Total Floor Area -SF,Year Built,ENERGY STAR Score,ENERGY STAR Certification - Eligibility,Latitude,Longitude,Electricity Use - Grid Purchase (kBtu),Office - Computer Density,Office - Weekly Operating Hours,Office - Worker Density,Multi Fami Max Floors,Multi Fami Resid Units,Multi Fami Area Cooled,Hotel Room Density,Hotel Worker Density,Hotel Perecent Area Cooled,Electricity Use - Grid Purchase (kwh)
0,7365,Manhattan,Office,686902.0,1982,NaN,No,40.756631,-73.982826,43525083.1,2.60711,90.00,2.45375,NaN,NaN,NaN,NaN,NaN,NaN,12752.849348
1,8139,Manhattan,Office,384432.0,1956,NaN,No,40.750698,-73.974306,16502793.1,4.29072,65.00,3.87549,NaN,NaN,NaN,NaN,NaN,NaN,4835.318378
2,8604,Manhattan,Financial Office,572369.0,1989,NaN,No,40.758310,-73.982504,24561685.2,1.35702,65.42,1.35702,NaN,NaN,NaN,NaN,NaN,NaN,7196.573764
3,8841,Manhattan,Office,362578.0,1961,NaN,No,40.753074,-73.972753,18881259.4,3.05200,70.00,2.77454,NaN,NaN,NaN,NaN,NaN,NaN,5532.209004
4,11809,Manhattan,Office,1601977.0,1999,NaN,No,40.756181,-73.986244,104995953.2,4.23490,98.00,4.30107,NaN,NaN,NaN,NaN,NaN,NaN,30763.814288


In [91]:
df_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12014 entries, 0 to 13132
Data columns (total 20 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Property Id                              12014 non-null  int64  
 1   City                                     12014 non-null  object 
 2   Primary Property Type                    12014 non-null  object 
 3   Total Floor Area -SF                     11970 non-null  float64
 4   Year Built                               12014 non-null  int64  
 5   ENERGY STAR Score                        2017 non-null   float64
 6   ENERGY STAR Certification - Eligibility  12014 non-null  object 
 7   Latitude                                 11332 non-null  float64
 8   Longitude                                11332 non-null  float64
 9   Electricity Use - Grid Purchase (kBtu)   12014 non-null  float64
 10  Office - Computer Density                2173 

In [ ]:
#Not available in "total floor area-sf"
#CHANGE TOTAL FLOOR AREA INTO INT
#CHANGE ENERGY SCORE INTO INT
#3 CHARA FOR OFFICE INTO FLOAT 64
#3 CHARA FOR MULTI FAMY BLDG INTO FLOAT 64
#3 CHARA FOR HOSPITAL INTO FLOAT 64
#CHANGE ALL "NOT AVAILABLE" INTO 'NAN'


In [96]:
df_work.value_counts(df_work['ENERGY STAR Certification - Eligibility'])

ENERGY STAR Certification - Eligibility
No     11738
Yes      276
dtype: int64

In [97]:
#df_process is the df that data inside are processed, IE NaN value will be filled or replaced
df_process = df_work

In [118]:
#Change to be made including following:
#1, Energy Star Score: NaN change into 50, which is national median
#2, Make a new col = 'Distance to Central Park' using "Latitude" & 'Longitude'
#3, Fill NaN value in Distance to Central Pak with average 
df_process['ENERGY STAR Score']=df_process['ENERGY STAR Score'].fillna(50)
df_process['Distance to Central Park'] = (abs(df_process['Latitude'])- 40.4712) + (abs(df_process['Longitude'])- 73.9665)**2
distance_avg = df_process['Distance to Central Park'].mean()


In [119]:
df_process['Distance to Central Park'] = df_process['Distance to Central Park'].fillna(distance_avg)

In [137]:
df_process['Year to now'] = abs(df_process['Year Built'])- 2020

In [147]:
#fill floor area NaN value as average
floorarea_avg = df_process['Total Floor Area -SF'].mean()
df_process['Total Floor Area -SF'] = df_process['Total Floor Area -SF'].fillna(distance_avg)

In [148]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12014 entries, 0 to 13132
Data columns (total 23 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Property Id                              12014 non-null  int64  
 1   City                                     12014 non-null  object 
 2   Primary Property Type                    12014 non-null  object 
 3   Total Floor Area -SF                     12014 non-null  float64
 4   Year Built                               12014 non-null  int64  
 5   ENERGY STAR Score                        12014 non-null  float64
 6   ENERGY STAR Certification - Eligibility  12014 non-null  object 
 7   Latitude                                 11332 non-null  float64
 8   Longitude                                11332 non-null  float64
 9   Electricity Use - Grid Purchase (kBtu)   12014 non-null  float64
 10  Office - Computer Density                2173 

In [149]:
#make test pipeline to make model for multi family bldg
#evaluating the final model

cols = [x for x in df_process.columns]
drop_list = ['Electricity Use - Grid Purchase (kwh)',
            'Electricity Use - Grid Purchase (kBtu)','Latitude','Longitude','Year Built']

for elem in drop_list:
    cols.remove(elem)

cols_x = cols
    
X= df_process[cols]
y= df_process['Electricity Use - Grid Purchase (kwh)']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of the complete data set:", X.shape)
print("Shape of the train data set:", X_train.shape)
print("Shape of the test data set:", X_test.shape)

Shape of the complete data set: (12014, 18)
Shape of the train data set: (9611, 18)
Shape of the test data set: (2403, 18)


In [ ]:
#process of data within dataline in the following:
#OneHotEncoder includes:('City','Primary Property Type')
#Scaler:('Total Floor Area -SF')
#linear Regression:('Year Built','ENERGY STAR Score')
#OrdinalEncoder include('ENERGY STAR Certification - Eligibility')

In [150]:
linear_col = ['Year to now','ENERGY STAR Score','Total Floor Area -SF']
Onehotencoder_col =['City','Primary Property Type']
Ordinalencoder_col = ['ENERGY STAR Certification - Eligibility']
Standscaler_col = ['Total Floor Area -SF']

transformer_common_cl = ColumnTransformer([
            ('Standscaler',StandardScaler(),Standscaler_col),
            ('Onehotencoder',OneHotEncoder(),Onehotencoder_col),
            ('Ordinalencoder',OrdinalEncoder(),Ordinalencoder_col)
])

In [151]:
class Ensemble_predictor(BaseEstimator, RegressorMixin):
    def __init__(self, raw_pred, residue_pred):
        self.raw_pred = raw_pred
        self.residue_pred = residue_pred
        
    def fit(self, X, y):
        self.raw_pred.fit(X, y)
        """
        Build a custom predictor that takes as an argument two other predictors. 
        It should use the first to fit the raw data and the second to fit the residuals of the first.
        """
        self.residue_pred.fit(X, y - self.raw_pred.predict(X))  #self.raw_pred.predict(X) is what's predicted by first predictor
        
        return self
    
    def predict(self, X):
        
        return self.raw_pred.predict(X) + self.residue_pred.predict(X)

In [152]:
Pipe_common_cl = Pipeline([('Transform', transformer_common_cl),
                            ('Predictor', Ensemble_predictor(Ridge(alpha = 10), 
                                  RandomForestRegressor(n_estimators = 100, max_depth=10, min_samples_leaf = 5)))
                   ])

In [164]:
#hand input paras
Pipe_common_cl.fit(X_train , y_train)

Pipeline(steps=[('Transform',
                 ColumnTransformer(transformers=[('Standscaler',
                                                  StandardScaler(),
                                                  ['Total Floor Area -SF']),
                                                 ('Onehotencoder',
                                                  OneHotEncoder(),
                                                  ['City',
                                                   'Primary Property Type']),
                                                 ('Ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['ENERGY STAR Certification '
                                                   '- Eligibility'])])),
                ('Predictor',
                 Ensemble_predictor(raw_pred=Ridge(alpha=10),
                                    residue_pred=RandomForestRegressor(max_depth=10,
                                                                       min_samples_leaf=5)))])

In [166]:
y_pred = Pipe_common_cl.predict(X_train)

In [167]:
r2_score(y_train, y_pred)

0.3613594176066145

In [157]:
transformer_common_cl.fit_transform(X_test)

<2403x120 sparse matrix of type '<class 'numpy.float64'>'
	with 7273 stored elements in Compressed Sparse Row format>

In [ ]:
# regroup the put cleaning of train data together.

df_2013 = pd.read_csv("raw data/Energy_and_Water_Data_Disclosure_for_Local_Law_84__Data_for_Calendar_Year_2013_.csv")

#pick columns to work with

cl_to_use_list = ['Property Id','City','Primary Property Type - Self Selected','Property Floor Area (Building(s)) (ft²)',
                 'Year Built','ENERGY STAR Score','ENERGY STAR Certification - Eligibility','Latitude','Longitude',
                  'Electricity Use - Grid Purchase (kBtu)',
                  'Office - Computer Density (Number per 1,000 ft²)','Office - Weekly Operating Hours','Office - Worker Density (Number per 1,000 ft²)',
                  'Multifamily Housing - Maximum Number of Floors','Multifamily Housing - Total Number of Residential Living Units','Multifamily Housing - Percent That Can Be Cooled',
                  'Hotel - Room Density (Number per 1,000 ft²)','Hotel - Worker Density (Number per 1,000 ft²)','Hotel - Percent That Can Be Cooled'
                 ]

df_2013_work = df_2013[cl_to_use_list]

#rename columns names

df_2013_work = df_2013_work.rename(columns = {'Property Floor Area (Building(s)) (ft²)':'Total Floor Area -SF',
                                              'Office - Computer Density (Number per 1,000 ft²)':'Office - Computer Density',
                                             'Primary Property Type - Self Selected':'Primary Property Type',
                                             'Hotel - Room Density (Number per 1,000 ft²)':'Hotel Room Density',
                                             'Hotel - Worker Density (Number per 1,000 ft²)':'Hotel Worker Density',
                                             'Hotel - Percent That Can Be Cooled':'Hotel Perecent Area Cooled',
                                             'Office - Worker Density (Number per 1,000 ft²)':'Office - Worker Density',
                                              'Multifamily Housing - Maximum Number of Floors':'Multi Fami Max Floors',
                                              'Multifamily Housing - Total Number of Residential Living Units':'Multi Fami Resid Units',
                                              'Multifamily Housing - Percent That Can Be Cooled':'Multi Fami Area Cooled'
                                             })

#turn the energy use (Y) col into kwh UNIT
df_2013_work = df_2013_work[df_2013_work['Electricity Use - Grid Purchase (kBtu)'] != 'Not Available']
df_2013_work['Electricity Use - Grid Purchase (kBtu)'] = pd.to_numeric(df_2013_work['Electricity Use - Grid Purchase (kBtu)'], errors='coerce')
df_2013_work['Electricity Use - Grid Purchase (kwh)'] = df_2013_work['Electricity Use - Grid Purchase (kBtu)']*0.000293

#turn the train col into wording and numeric columns

df_work = df_2013_work

# to change all columns with "not available" into 'nan'
cl_not_available = ['ENERGY STAR Score', 'Electricity Use - Grid Purchase (kwh)',
                    'Total Floor Area -SF','Office - Computer Density','Office - Weekly Operating Hours','Office - Worker Density',
                   'Multi Fami Max Floors','Multi Fami Resid Units','Multi Fami Area Cooled',
                   'Hotel Room Density','Hotel Worker Density','Hotel Perecent Area Cooled']


def turn_notavailable(x):

    if x == 'Not Available':
        x=np.nan
    return x

for col in cl_not_available:
    df_work[col] = df_work[col].apply(turn_notavailable)
    

# to change all columns with dtype 'object' into 'int' or 'float'
cl_to_numeric = ['Total Floor Area -SF','ENERGY STAR Score',
                    'Office - Computer Density','Office - Weekly Operating Hours','Office - Worker Density',
                   'Multi Fami Max Floors','Multi Fami Resid Units','Multi Fami Area Cooled',
                   'Hotel Room Density','Hotel Worker Density','Hotel Perecent Area Cooled']

for col in cl_to_numeric:
    df_work[col] = pd.to_numeric(df_work[col], errors='coerce')
    


In [ ]:
# regroup the put pipeline to train home type building together
df_process = df_work

"""#Change to be made including following:
#1, Energy Star Score: NaN change into 50, which is national median
#2, Make a new col = 'Distance to Central Park' using "Latitude" & 'Longitude'
#3, Fill NaN value in Distance to Central Pak with average """
df_process['ENERGY STAR Score']=df_process['ENERGY STAR Score'].fillna(50)
df_process['Distance to Central Park'] = (abs(df_process['Latitude'])- 40.4712) + (abs(df_process['Longitude'])- 73.9665)**2
distance_avg = df_process['Distance to Central Park'].mean()
df_process['Distance to Central Park'] = df_process['Distance to Central Park'].fillna(distance_avg)

df_process['Multi Fami Max Floors'] = df_process['Multi Fami Max Floors'].fillna(df_process['Multi Fami Max Floors'].mean())
df_process['Multi Fami Resid Units'] = df_process['Multi Fami Resid Units'].fillna(df_process['Multi Fami Resid Units'].mean())
df_process['Multi Fami Area Cooled'] = df_process['Multi Fami Area Cooled'].fillna(df_process['Multi Fami Area Cooled'].mean())



df_process['Year to now'] = abs(df_process['Year Built'])- 2020


cols = [x for x in df_process.columns]
drop_list = ['Electricity Use - Grid Purchase (kwh)',
            'Electricity Use - Grid Purchase (kBtu)','Latitude','Longitude','Year Built']

for elem in drop_list:
    cols.remove(elem)

home_cols = cols.remove('Office - Computer Density','Office - Weekly Operating Hours','Office - Worker Density',
                       'Hotel Room Density','Hotel Worker Density','Hotel Perecent Area Cooled')

office_cols = cols.remove('Multi Fami Max Floors','Multi Fami Resid Units','Multi Fami Area Cooled',
                       'Hotel Room Density','Hotel Worker Density','Hotel Perecent Area Cooled')

hotel_cols = cols.remove('Multi Fami Max Floors','Multi Fami Resid Units','Multi Fami Area Cooled',
                       'Office - Computer Density','Office - Weekly Operating Hours','Office - Worker Density')


#separate different columns for different type of buildings
X_home_train= df_process[home_cols]
X_hotel_train= df_process[hotel_cols]
X_office_train= df_process[office_cols]

y_train= df_process['Electricity Use - Grid Purchase (kwh)']

home_linear_col = ['Year to now','ENERGY STAR Score','Total Floor Area -SF','Multi Fami Max Floors','Multi Fami Resid Units','Multi Fami Area Cooled']
Onehotencoder_col =['City','Primary Property Type']
Ordinalencoder_col = ['ENERGY STAR Certification - Eligibility']
Standscaler_col = ['Total Floor Area -SF']

transformer_common_cl = ColumnTransformer([
            ('Standscaler',StandardScaler(),Standscaler_col),
            ('Onehotencoder',OneHotEncoder(),Onehotencoder_col),
            ('Ordinalencoder',OrdinalEncoder(),Ordinalencoder_col)
])

class Ensemble_predictor(BaseEstimator, RegressorMixin):
    def __init__(self, raw_pred, residue_pred):
        self.raw_pred = raw_pred
        self.residue_pred = residue_pred
        
    def fit(self, X, y):
        self.raw_pred.fit(X, y)
        """
        Build a custom predictor that takes as an argument two other predictors. 
        It should use the first to fit the raw data and the second to fit the residuals of the first.
        """
        self.residue_pred.fit(X, y - self.raw_pred.predict(X))  #self.raw_pred.predict(X) is what's predicted by first predictor
        
        return self
    
    def predict(self, X):
        
        return self.raw_pred.predict(X) + self.residue_pred.predict(X)
    
    
Home_bldg_model = Pipeline([('Transform', transformer_common_cl),
                            ('Predictor', Ensemble_predictor(Ridge(alpha = 10), 
                                  RandomForestRegressor(n_estimators = 100, max_depth=10, min_samples_leaf = 5)))
                   ])


Home_bldg_model.fit(X_home_train , y_train)
Home_filename = 'home_bldg_model.sav'
pickle.dump(Home_bldg_model, open(Home_filename, 'wb'))


Office_bldg_model.fit(X_office_train , y_train)
Office_filename = 'home_bldg_model.sav'
pickle.dump(Office_bldg_model, open(Office_filename, 'wb'))

Hotel_bldg_model.fit(X_hotel_train , y_train)
Hotel_filename = 'Hotel_bldg_model.sav'
pickle.dump(Hotel_bldg_model, open(Hotel_filename, 'wb'))